In [0]:
import tarfile
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

In [0]:
fname = "lenses.tgz"
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

In [0]:
cd lenses

/content/lenses


In [0]:
ls

no_sub/  sub/


In [0]:
import glob

In [0]:
sub_img = []
for img in glob.glob("sub/*.jpg"):
    n = cv2.imread(img)
    sub_img.append(n)

In [0]:
no_sub_img = []
for img in glob.glob("no_sub/*.jpg"):
    n = cv2.imread(img)
    no_sub_img.append(n)

In [0]:
len(no_sub_img)

5000

In [0]:
overall_dataset = []
for i in sub_img:
    overall_dataset.append((i,0))

for i in no_sub_img:
    overall_dataset.append((i,1))

In [0]:
import random
random.shuffle(overall_dataset)

In [0]:
data = [i[0] for i in overall_dataset]
labels = [i[1] for i in overall_dataset]

In [0]:
train_data = data[:7500]
train_labels = labels[:7500]

In [0]:
test_data = data[7500:]
test_labels = labels[7500:]

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        self.c1 = nn.Sequential(
                    nn.Conv2d(3,6,5),
                    nn.ReLU(inplace=True),
                    nn.BatchNorm2d(6)
        )

        self.c2 = nn.Sequential(
                   nn.MaxPool2d(2,2),
                   nn.Conv2d(6,16,5),
                   nn.ReLU(inplace=True),
                   nn.BatchNorm2d(16) 
        )

        self.fc = nn.Sequential(
                    nn.Linear(76176, 1200),
                    nn.ReLU(inplace=True),
                    nn.Linear(1200,200),
                    nn.ReLU(inplace=True),
                    nn.Linear(200,2)

        )

    def forward(self,x):
        x = self.c1(x)
        x = self.c2(x)
        x = x.view(-1,76176)
        x = self.fc(x)
        return x

In [0]:
net = Net().cuda()

#TRAINING TIME

In [0]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum = 0.9)

In [0]:
for epoch in range(5):
    for d in range(75):
        batch_train_data_curr = torch.from_numpy(np.array(train_data[d*100:d*100+100]).reshape(-1,3,150,150)).type(torch.FloatTensor).cuda()
        batch_train_label_curr = torch.from_numpy(np.array(train_labels[d*100:d*100+100])).type(torch.LongTensor).cuda()

        optimizer.zero_grad()

        outputs = net(batch_train_data_curr)

        loss = criterion(outputs, batch_train_label_curr)

        loss.backward()
        optimizer.step()

        if(d%10 == 0):
            print("EPOCH IS {}\n AND CURRENT LOSS IS {}/n".format(epoch, loss.item()))

EPOCH IS 0
 AND CURRENT LOSS IS 0.6906511187553406/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6920977830886841/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6963557600975037/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6675735712051392/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6556903123855591/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6457473635673523/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6644276976585388/n
EPOCH IS 0
 AND CURRENT LOSS IS 0.6487205028533936/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.6603279709815979/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.6323608160018921/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.6480584740638733/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.6064150929450989/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.6085724830627441/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.5798658132553101/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.6136316061019897/n
EPOCH IS 1
 AND CURRENT LOSS IS 0.5978912711143494/n
EPOCH IS 2
 AND CURRENT LOSS IS 0.6038119792938232/n
EPOCH IS 2
 AND CURRENT LOSS IS 0.5746724605560303/n
EPOCH IS 2
 AND CURRENT LOSS IS 0.590006351470

In [0]:
count = 0
for z,i in enumerate(test_data):
    j = torch.from_numpy(i.reshape(-1,3,150,150)).type(torch.FloatTensor).cuda()
    a = net(j)
    _,b = torch.max(a,1)
    if(b.item()==test_labels[z]):
        count +=1
    

In [0]:
count /= 2500

In [0]:
count

0.7132